In [ ]:
pip install pandas

In [ ]:
import pandas as pd

In [ ]:
pwd

In [ ]:
cd C:\Users\user\Desktop\Megha- MTECH

In [ ]:
ds = pd. read_csv("/content/drive/MyDrive/Colab Notebooks/Sample model/new_bert.csv")

In [ ]:
ds

,cleaned_text,Analysis
0,Australia to Manufacture Covid Vaccine and giv...,0
1,CoronavirusVaccine CoronaVaccine CovidVaccine...,0
2,Deaths due to COVID in Affected Countries Read...,1
3,Team Subhashree subhashreesotwe iamrajchoco S...,0
4,michellegrattan ConversationEDU This is what ...,2
...,...,...
373613,BioNTech License shall mean the non exclusive...,0
373614,What if there was something that reduced Covid...,0
373615,Pro EU FBPE loons wetting more beds DFS must h...,1
373616,CovidVaccine Six African countries to begin m...,0


In [ ]:
ds.Analysis.value_counts()

0    167749
2    150759
1     55110
Name: Analysis, dtype: int64

0: positive
1:Negative
2:Neutral

In [ ]:
possible_labels = ds.Analysis.unique()

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
ds['label'] = ds.Analysis.replace(label_dict)

In [ ]:
ds.head()

,cleaned_text,Analysis,label
0,Australia to Manufacture Covid Vaccine and giv...,0,0
1,CoronavirusVaccine CoronaVaccine CovidVaccine...,0,0
2,Deaths due to COVID in Affected Countries Read...,1,1
3,Team Subhashree subhashreesotwe iamrajchoco S...,0,0
4,michellegrattan ConversationEDU This is what ...,2,2


In [ ]:
pip install sklearn

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(ds.index.values, 
                                                  ds.label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=ds.label.values)

In [ ]:
ds['data_type'] = ['not_set']*ds.shape[0]

In [ ]:
ds.loc[X_train, 'data_type'] = 'train'
ds.loc[X_val, 'data_type'] = 'val'

In [ ]:
ds.groupby(['Analysis', 'label', 'data_type']).count()


cleaned_text
Analysis label data_type              
0        0     train            142587
               val               25162
1        1     train             46843
               val                8267
2        2     train            128145
               val               22614

In [ ]:
pip install transformers

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 60.0 MB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 
     |████████████████████████████████| 6.6 MB 27.4 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers

In [ ]:
pip install torch

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

In [ ]:
!nvidia-smi

Tue Apr 19 20:20:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu112

In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = torch.device('cuda')

In [ ]:
torch.cuda.current_device()

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    ds[ds.data_type=='train'].cleaned_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    ds[ds.data_type=='val'].cleaned_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(ds[ds.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(ds[ds.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

In [ ]:
len(dataset_val)

setting up bert pretrained model

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
to_device(model,device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import keras

In [ ]:
model.summary()

AttributeError: ignored

 creating Data loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

setting optimizer and schduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

Performance metrics

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
accuracy_list = []
c_x = 0
c_y = 0

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        x = len(y_preds[y_preds==label])
        y = len(y_true)
        print(f'Accuracy: {x}/{y}\n')
        global c_x, c_y
        c_x = c_x + x
        c_y = c_y + y
        
    accuracy_list.append(c_x)
    accuracy_list.append(c_y)

creating training loop

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from ipywidgets import FloatProgress

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
     
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/9925 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed


Epoch 1
Training loss: 0.18531942274386265
Validation loss: 0.10333345195252744
F1 Score (Weighted): 0.9710009145966103


Epoch 2:   0%|          | 0/9925 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.08091215213856916
Validation loss: 0.07704859860830798
F1 Score (Weighted): 0.9781783297435236


Epoch 3:   0%|          | 0/9925 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.05706758345644722
Validation loss: 0.07459448934745831
F1 Score (Weighted): 0.9797137529351104


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 24815/25162

Class: 1
Accuracy: 7934/8267

Class: 2
Accuracy: 22156/22614



In [ ]:
print('Overall Accuracy:\n{0}/{1} \n{2}'.format(accuracy_list[0], accuracy_list[1], accuracy_list[0]/accuracy_list[1]))

Overall Accuracy:
54905/56043 
0.9796941634102385


In [ ]:
torch.save(model.state_dict(), './Megha_BERT_full.pth')

In [ ]:
output_model = './Megha_BERT_full_1.pth'
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)

In [ ]:
checkpoint = torch.load(output_model)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

<a href="./Megha_BERT.pth"> Download File </a>

<a href="./Megha_BERT_full.pth"> Download File </a>

<a href="./Megha_BERT_full_1.pth"> Download File </a>

<a href="./finetuned_BERT_epoch_3.model"> Download File </a>

In [ ]:
model.eval()

In [ ]:
model1.load_state_dict(torch.load('./Megha_BERT_full.pth'))

In [ ]:
def evaluate1(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
_, predictions, true_vals = evaluate1(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/Code/BERT/BERT/Megha_BERT_full_1.pth', map_location=torch.device('cpu'))

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)